# 写在最前

请记住，在本章的内容中，“架构”指的是模型的结构，而 "checkpoint" 是给定架构的权重（也就是预训练模型的权重，也可指代预训练模型本身）。

例如，“BERT” 是一种架构，而 “google-bert/bert-base-uncased” 是一个 checkpoint。“模型”则是一个通用术语，可以指代架构或checkpoint。

在 `🤗 Transformers` 的示例代码中，你通过替换 "model" 参数的值来选择不同的模型。

在这个教程中，将会学习如何：

1. 加载预训练的分词器（tokenizer）
2. 加载预训练的图像处理器(image processor)
3. 加载预训练的特征提取器(feature extractor)
4. 加载预训练的处理器(processor)
5. 加载预训练的模型。

# 使用AutoClass加载预训练实例

由于存在许多不同的 Transformer 架构，因此为 checkpoint 创建一个可用的架构可能会具有挑战性。

**通过 `AutoClass` 可以自动推断并从给定的 checkpoint 加载正确的架构**, 这也是体现 🤗 Transformers 易于使用、简单且灵活的核心部分。

**如果你的代码适用于一个 checkpoint，它将适用于另一个 checkpoint**，只要它们是为了相似的任务进行训练的，即使它们的架构有所不同。

`from_pretrained()` 方法允许你快速加载任何架构的预训练模型，不需要花费时间和精力从头开始训练模型。

## AutoTokenizer

几乎所有的 NLP 任务都以 `tokenizer` 开始。Tokenizer 能够将你的输入转换为模型可以理解和处理的格式(即模型正确的输入格式)。

使用 `AutoTokenizer.from_pretrained()` 加载 tokenizer：

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

然后按照如下方式对输入进行分词：

In [ ]:
sequence = "In a hole in the ground there lived a hobbit."
print(tokenizer(sequence))

## AutoImageProcessor

对于视觉任务，`ImageProcessor` 能够将图像处理成正确的输入格式。

这里使用`AutoImageProcessor.from_pretrained()`加载 Image Processor：

In [ ]:
from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")

## AutoFeatureExtractor

对于音频任务, `Feature Extractor` 将音频信号处理成正确的输入格式。

这里使用`AutoFeatureExtractor.from_pretrained()`加载 Feature Extractor：

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(
    "ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition"
)

## AutoProcessor

对于多模态任务只需要一个 `Processor` 就能够将两种类型的预处理工具结合起来。

例如，LayoutLMV2 模型需要一个 image processor 来处理图像和一个 tokenizer 来处理文本；Processor 会将两者结合起来。

使用`AutoProcessor.from_pretrained()`加载 Processor：

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("microsoft/layoutlmv2-base-uncased")

## AutoModel

`AutoModelFor*` 类能够加载给定任务类型的预训练模型（[支持的任务列表](https://huggingface.co/docs/transformers/main/en/model_doc/auto)）。

例如，使用 `AutoModelForSequenceClassification.from_pretrained()` 加载用于序列分类的模型：

In [ ]:
# PyTorch
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased")

可以使用相同的 checkpoint 来为不同的任务加载模型架构：

In [ ]:
# PyTorch
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("distilbert/distilbert-base-uncased")

对于 PyTorch 模型，`from_pretrained()`方法在加载预训练模型时，通常使用`torch.load()`，它内部使用的是不安全的 `pickle`， pickle 加载不可信的数据可能存在安全风险（如代码执行漏洞）。

永远不要加载那些来自不可信来源或可能被篡改的模型。建议对于托管在 Hugging Face Hub 上的公共模型，因为每次提交都会进行恶意软件扫描，在一定程度上缓解了安全风险。

建议使用[使用 GPG 进行签名提交验证](https://huggingface.co/docs/hub/security-gpg#signing-commits-with-gpg)等最佳实践来进一步确保安全性，可参阅[Hub 文档](https://huggingface.co/docs/hub/security)以了解最佳实践。

**TensorFlow 和 Flax 在加载 checkpoints 不使用 pickle ，不存在由 pickle 引起的安全问题。** 

在 PyTorch 中可以通过 `from_pretrained()` 方法的 `from_tf` 和 `from_flax` 参数来加载 TensorFlow 和 Flax 的 checkpoints，进而绕过 torch.load() 使用 pickle 的安全风险。

### 实际操作示例

通过加载 TensorFlow和 Flax 的 checkpoints，可以避免使用 torch.load()及其pickle的安全风险。

假设你想要加载一个在 Hugging Face Hub 上托管的 TensorFlow 模型到 PyTorch 中，可以这样做：

In [ ]:
from transformers import AutoModel

# 加载TensorFlow模型，并将 TensorFlow 模型转换为 PyTorch 模型
pt_model = AutoModel.from_pretrained("bert-base-uncased", from_tf=True)

类似地，如果你要加载一个 Flax 模型到 PyTorch 中：

In [ ]:
from transformers import AutoModel

# 加载 Flax 模型，并将 Flax 模型转换为 PyTorch 模型
pt_model = AutoModel.from_pretrained("bert-base-uncased", from_flax=True)

一般来说，我们不建议使用通用的`AutoModel`来加载预训练模型，而是使用`AutoTokenizer`类和`AutoModelFor*`类来加载预训练的模型实例，因为这样可以确保加载的是正确的架构。

在下一个[教程](../docs/4_preprocess_data.ipynb)中，学习如何使用新加载的 tokenizer, image processor, feature extractor 和 processor对数据集进行预处理以进行微调。